### Second Project

In [3]:
using Plots, ROCAnalysis, GLM, MLBase, Lathe, Distributions, LaTeXStrings, CSV, DataFrames, StatsBase, Missings, XLSX
using IJulia: readprompt

#### Q1

In [8]:
df = DataFrame(XLSX.readtable("crypto_currency.xlsx", "Sheet1")...)

,name,ticket,market cap,last price($)
,Any,Any,Any,Any
1,Bitcoin,BTC,750B,39K
2,Ethereum,ETH,360B,3K
3,Ripple,XRP,50B,0.7
4,Solana,SOL,34B,100
5,Terra,LUNA,32B,91
6,Cardano,ADA,30B,0.9
7,Avalanche,AVAX,20B,73
8,Polkadot,DOT,19B,19
9,Dogecoin,DOGE,18B,0.13


In [19]:
names(df)

4-element Vector{String}:
 "name"
 "ticket"
 "market cap"
 "last price(\$)"

In [110]:
selected_rows = findall(x->x ∈ sort(df."market cap", rev=true)[1:5], df."market cap")
top_five=df[selected_rows,:]

,name,ticket,market cap,last price($)
,Any,Any,Any,Any
1,Bitcoin,BTC,750B,39K
2,Ethereum,ETH,360B,3K
3,Ripple,XRP,50B,0.7
4,Solana,SOL,34B,100
5,Terra,LUNA,32B,91


In [108]:
for i in top_five."last price(\$)"
    if typeof(i) == String && contains(i, "K") == true
        replace!(top_five."last price(\$)", i=>parse(Int, chop(i))*1000)
    end
end
top_five

,name,ticket,market cap,last price($)
,Any,Any,Any,Any
1,Bitcoin,BTC,750B,-20.9895
2,Ethereum,ETH,360B,0.0474742
3,Ripple,XRP,50B,0.0471464
4,Solana,SOL,34B,0.0473507
5,Terra,LUNA,32B,0.0475091


In [101]:
sum(top_five."last price(\$)")

42191.7

##### *sclaing data*
$$
x := \frac{x - \mu}{\sigma}
$$

In [107]:

replace!(x -> (x - mean(top_five."last price(\$)"))/var(top_five."last price(\$)"), top_five."last price(\$)")
top_five

,name,ticket,market cap,last price($)
,Any,Any,Any,Any
1,Bitcoin,BTC,750B,-20.9895
2,Ethereum,ETH,360B,0.0474742
3,Ripple,XRP,50B,0.0471464
4,Solana,SOL,34B,0.0473507
5,Terra,LUNA,32B,0.0475091


## Mainnnn


In [44]:
push!(vec(convert(Array, select!(CSV.read("LUNA-USD.csv", DataFrame), "Close"))), "null")

366-element Vector{String15}:
 "0.007437"
 "0.006072"
 "0.006019"
 "0.006317"
 "0.006501"
 "0.006713"
 "0.006002"
 "0.005224"
 "0.003865"
 "0.004214"
 "0.004325"
 "0.004612"
 "0.004280"
 ⋮
 "0.004023"
 "0.003965"
 "0.003905"
 "0.003747"
 "0.003636"
 "0.005584"
 "1.192089"
 "null"
 "null"
 "null"
 "null"
 "null"

In [45]:
df = DataFrame("Bitcoin" => vec(convert(Array, select!(CSV.read("BTC-USD.csv", DataFrame), "Close"))),
     "Ethereum"=> vec(convert(Array, select!(CSV.read("ETH-USD.csv", DataFrame), "Close"))), 
     "Ripple"=> vec(convert(Array, select!(CSV.read("XRP-USD.csv", DataFrame), "Close"))),
     "Solana"=> vec(convert(Array, select!(CSV.read("SOL-USD.csv", DataFrame), "Close"))), 
     "Terra"=> push!(vec(convert(Array, select!(CSV.read("LUNA-USD.csv", DataFrame), "Close"))), "null"),
     "Cardano"=> vec(convert(Array, select!(CSV.read("ADA-USD.csv", DataFrame), "Close"))), 
     "Avalanche"=> vec(convert(Array, select!(CSV.read("AVAX-USD.csv", DataFrame), "Close"))), 
     "Polkadot"=> vec(convert(Array, select!(CSV.read("DOT-USD.csv", DataFrame), "Close"))),
     "Dogecoin"=> vec(convert(Array, select!(CSV.read("DOGE-USD.csv", DataFrame), "Close"))), 
     "Polygon"=> vec(convert(Array, select!(CSV.read("MATIC-USD.csv", DataFrame), "Close"))))

,Bitcoin,Ethereum,Ripple,Solana,Terra,Cardano,Avalanche,Polkadot,Dogecoin
,Float64,Float64,Float64,Float64,String15,Float64,Float64,String15,Float64
1,46456.1,3587.51,1.45261,47.2008,0.007437,2.30911,36.5188,42.631512,0.514895
2,43537.5,3282.4,1.4963,46.8632,0.006072,2.03223,33.6623,38.710159,0.487892
3,42909.4,3380.07,1.59073,55.911,0.006019,2.00366,36.8222,40.645355,0.476115
4,37002.4,2460.68,1.07031,35.1072,0.006317,1.48252,23.3839,25.227142,0.333123
5,40782.7,2784.29,1.18554,44.7427,0.006501,1.81047,25.1969,29.150570,0.400194
6,37304.7,2430.62,1.00698,38.8129,0.006713,1.55228,21.8018,24.463970,0.359382
7,37536.6,2295.71,0.906317,31.3156,0.006002,1.46362,18.8511,22.014524,0.342371
8,34770.6,2109.58,0.797015,24.6887,0.005224,1.32544,15.7562,18.029442,0.308071
9,38706.0,2643.59,0.992835,31.2836,0.003865,1.54949,19.4109,23.199089,0.366162


### Q2